<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>
<hr>




<p align="center"><h1 align="center">Quick Start: Flower Image Classification Tutorial</h1> 

<hr>

<h3 align="center">(Deploy model to an AI Model Share Model Playground REST API<br> and Web Dashboard in five easy steps...)</h3></p>
<p align="center"><img width="100%" src="https://aimodelsharecontent.s3.amazonaws.com/aimstutorialsteps.gif" /></p>



## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*


In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade


In [ ]:
# Generate credentials file
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

## **Set up Environment**

Use your credentials file to set your credentials for all aimodelshare functions. 

In [2]:
# Set credentials 
import aimodelshare as ai
from aimodelshare.aws import set_credentials

set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [ ]:
# Get materials for tutorial
import aimodelshare as ai
keras_model, y_train_labels = ai.import_quickstart_data("flowers")

## **(1) Preprocessor Function & Setup**

### **Write a Preprocessor Function**


> ###   Preprocessor functions are used to preprocess data into the precise data your model requires to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For image prediction models users should minimally include function inputs for an image filepath and values to reshape the image height and width.*  



In [5]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(image_filepath, shape=(192, 192)):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

## **(2) Train Model Using tf.keras (or Your Preferred ML Library)**

### Keras **Convolutional Neural Network**

In [6]:
#Here is a pre-trained SqueezeNet keras_model, but you could also train your own model after preprocessing data with your preprocessor function.
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 192, 192, 64)      256       
_________________________________________________________________
dense_3 (Dense)              (None, 192, 192, 64)      4160      
_________________________________________________________________
dense_4 (Dense)              (None, 192, 192, 64)      4160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2359296)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 11796485  
Total params: 11,805,061
Trainable params: 11,805,061
Non-trainable params: 0
_________________________________________________________________


## **(3) Save Preprocessor**
### Saves preprocessor function to "preprocessor.zip" file

In [7]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [8]:
#  Now let's import and test the preprocessor function to see if it is working...

import aimodelshare as ai
prep=ai.import_preprocessor("preprocessor.zip")

prep("quickstart_materials/example_data/100080576_f52e8ee070_n.jpg").shape

(1, 192, 192, 3)

## **(4) Save Keras Model to Onnx File Format**


In [9]:
# Save tf.keras model (or any tensorflow model) to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(keras_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## **(5) Create your Model Playground and Deploy REST API/ Live Web-Application**




In [10]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata_filepath = "quickstart_materials/example_data" 

In [11]:
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class
myplayground=ModelPlayground(model_type="image", classification=True, private=False)

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)

myplayground.deploy(model_filepath, preprocessor_filepath, y_train_labels, exampledata_filepath) 

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):Flower Image Classifier
Model Description (Explain what your model does and 
 why end-users would find your model useful):Takes flower images and classifies them into one of five categories.
Model Key Words (Search categories that describe your model, separated with commas):flower, image, classification
   
Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 88 seconds. 
 Playground Url: "https://dfwj72k5hh.execute-api.us-east-1.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can make predictions with the Dashboard and access example code from the Programmatic tab.
https://www

## **Use your new Model Playground!**

Follow the link in the output above to:
- Generate predictions with your interactive web dashboard
- Access example code in Python, R, and Curl

## **Or, follow the rest of the tutorial to create a competition for your Model Playground and:** 
- Access verified model performance metrics 
- Upload multiple models to a leaderboard 
- Easily compare model performance & structure 

## **Part 2: Create a Competition**

-------

After deploying your Model Playground, you can now create a competition. 

Creating a competition allows you to:
1. Verify the model performance metrics on aimodelshare.org
2. Submit models to a leaderboard
3. Grant access to other users to submit models to the leaderboard
4. Easily compare model performance and structure 

In [12]:
#Download competition flower image data and a list of y_test labels to use to evaluate model prediction submissions
import aimodelshare as ai
keras_model_2, y_test_labels = ai.import_quickstart_data("flowers", "competition")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


In [13]:
# Create list of authorized participants for competition
# Note that participants should use the same email address when creating modelshare.org account

emaillist=["emailaddress1@email.com", "emailaddress2@email.com"]


In [ ]:
# Create Competition
myplayground.create_competition(data_directory='flower_competition_data', 
                                y_test = y_test_labels, email_list=emaillist) 

In [15]:
#Instantiate Competition before submitting models.

#--Note: If you start a new session, the first argument should be the Model Playground url in quotes. 
#--e.g.- mycompetition= ai.Competition("https://123456.execute-api.us-east-1.amazonaws.com/prod/m)
#See Model Playground "Compete" tab for example model submission code.

mycompetition= ai.Competition(myplayground.playground_url)

In [ ]:
# Add, remove, or completely update authorized participants for competition later
emaillist=["emailaddress3@email.com"]

mycompetition.update_access_list(email_list=emaillist,update_type="Add")


Submit Models

In [17]:
#Authorized users can submit new models after setting credentials using modelshare.org username/password

apiurl=myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m

from aimodelshare.aws import set_credentials
set_credentials(apiurl=apiurl)


AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [18]:
# Preprocess X_test image data to generate predictions from models 
import numpy as np

file_names = [('flower_competition_data/test_images/' + str(i) + '.jpg') for i in range(1, 735)]

preprocessed_image_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

# One-hot encode y_train labels (y_train.columns used to generate prediction labels below)
import pandas as pd
y_train = pd.get_dummies(y_train_labels)

In [ ]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=keras_model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

In [20]:
# Have a look at architecture for model two
keras_model_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 192, 192, 32)      416       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 192, 192, 32)      4128      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 16)        2064      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 16)        1040      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 16)        0

In [21]:
# Save Model 2 to .onnx file

# Save tf.keras model to ONNX file
onnx_cnn2 = model_to_onnx(keras_model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_cnn2.SerializeToString()) 

In [ ]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
prediction_column_index=keras_model_2.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Get Leaderboard

In [23]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,conv2d_layers,dense_layers,dropout_layers,flatten_layers,maxpooling2d_layers,relu_act,softmax_act,loss,optimizer,model_config,memory_size,username,version
0,64.17%,63.68%,64.27%,63.72%,keras,False,True,Sequential,11,1851153,4.000000,2,2.000000,1,2.000000,5,1,function,RMSprop,"{'name': 'sequential', 'layers...",303848,mikedparrott,2
1,48.50%,47.44%,49.65%,46.96%,keras,False,True,Sequential,5,11805061,nan,4,nan,1,nan,3,1,function,RMSprop,"{'name': 'sequential_1', 'laye...",205640,mikedparrott,1


Compare Models

In [24]:
# Compare two or more models (Experimental, Git-like Diffs for Model Architectures)
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,Dense,"[None, 192, 192, 64]",256.000000,Conv2D,"[None, 192, 192, 32]",416
1,Dense,"[None, 192, 192, 64]",4160.000000,Conv2D,"[None, 192, 192, 32]",4128
2,Dense,"[None, 192, 192, 64]",4160.000000,MaxPooling2D,"[None, 96, 96, 32]",0
3,Flatten,"[None, 2359296]",0.000000,Conv2D,"[None, 96, 96, 16]",2064
4,Dense,"[None, 5]",11796485.000000,Conv2D,"[None, 96, 96, 16]",1040
5,None,None,nan,MaxPooling2D,"[None, 48, 48, 16]",0
6,None,None,nan,Dropout,"[None, 48, 48, 16]",0
7,None,None,nan,Flatten,"[None, 36864]",0
8,None,None,nan,Dense,"[None, 50]",1843250
9,None,None,nan,Dropout,"[None, 50]",0


#### Check structure of y test data 
(This helps users understand how to submit predicted values to leaderboard)

In [25]:
mycompetition.inspect_y_test()

{'class_balance': {'daisy': 126,
  'dandelion': 180,
  'roses': 128,
  'sunflowers': 140,
  'tulips': 160},
 'class_labels': ['daisy', 'dandelion', 'sunflowers', 'roses', 'tulips'],
 'label_dtypes': {"<class 'str'>": 734},
 'y_length': 734,
 'ytest_example': ['daisy', 'sunflowers', 'roses', 'dandelion', 'roses']}

## **Part 3: Maintaining your Model Playground**

-------

Update Runtime model

*Use this function to 1) update the prediction API behind your Model Playground with a new model, chosen from the leaderboard and 2) verify the modelperformance metrics in your model playground*

In [26]:
myplayground.update_runtime_model(model_version=2)

Runtime model & preprocessor for api: https://dfwj72k5hh.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 2.

Model metrics are now updated and verified for this model playground.


Delete Deployment 

*Use this function to delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models*

In [ ]:
myplayground.delete_deployment()

Running this function will permanently delete all resources tied to this deployment, 
 including the eval lambda and all models submitted to the model competition.

To confirm, type 'permanently delete':permanently delete


'API deleted successfully.'